In [1]:
import torch
import torch.nn as nn
import numpy as np

import datasets.datasets as dtset
import utils.losses as losses

from utils.weakener import Weakener
from models.model import MLP

from utils.trainig_testing import train_and_evaluate

In [2]:
Data = dtset.Torch_Dataset('mnist', batch_size = 16)
Weak = Weakener(Data.num_classes)
Weak.generate_M(model_class='pll',pll_p=0.2)

In [3]:
train_X,train_y,test_X,test_y =  Data.get_data()
Weak.generate_weak(train_y) #z and w


(tensor([335, 529,  42,  ...,  47,   9,   9], dtype=torch.int32),
 tensor([[0., 1., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.]], dtype=torch.float64))

In [4]:
'''
import pickle
Dataset = [Data,Weak]
f = open("Experimental_results/Datasets.pkl","wb")
pickle.dump(Dataset,f)
f.close()
'''

'\nimport pickle\nDataset = [Data,Weak]\nf = open("Experimental_results/Datasets.pkl","wb")\npickle.dump(Dataset,f)\nf.close()\n'

In [5]:
Weak.z.shape

torch.Size([60000])

In [6]:
#Depending the model we are about to feed we will need either weak labels or virtual labels. 

In [7]:
Weak.virtual_labels(p=None, optimize = False, convex = True) #This is to create Virtual Labels


c:\Users\danibacaicoa\vscode_projects\Learning_from_weak_labels\.venv\lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


In [8]:
Data.include_weak(Weak.z)
#Data.include_virtual(Weak.v)

In [9]:
trainloader,testloader = Data.get_dataloader(weak_labels='weak')

In [10]:
examples = next(iter(trainloader))
for k in examples:
    print(k)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([  3, 339, 531, 353, 512,  19,  27, 431, 303, 167,  60, 448,   0, 547,
        521, 261], dtype=torch.int32)
tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0

In [11]:
Weak.V_matrix(Data.num_classes)

array([[0.1796228 , 0.06898198, 0.12434957, ..., 0.09171096, 0.10871832,
        0.07654192],
       [0.14023764, 0.15947086, 0.11958131, ..., 0.08669024, 0.16870691,
        0.12970188],
       [0.14231886, 0.05226663, 0.05998836, ..., 0.06022358, 0.16111562,
        0.06627856],
       ...,
       [0.07704287, 0.0965763 , 0.13342503, ..., 0.10547777, 0.04821152,
        0.20792012],
       [0.01375156, 0.1331928 , 0.13435628, ..., 0.20184827, 0.1220006 ,
        0.13111119],
       [0.09522949, 0.09678306, 0.03010278, ..., 0.15785421, 0.15120496,
        0.00619227]])

In [12]:

loss = losses.FBLoss(Weak.M,Weak.V)
overall_results = {}
overall_models = {}
epochs = 5
for i in range(2):
    mlp = MLP(Data.num_features,[Data.num_features],Data.num_classes, dropout_p=0.5, bn=True, activation =  'gelu')
    optim = torch.optim.Adam(mlp.parameters(),lr=1e-2)
    mlp, results = train_and_evaluate(mlp,trainloader,testloader,optimizer=optim,loss_fn=loss,num_epochs=epochs,sound=1)
    overall_results[i] = results
    overall_models[i] = mlp  

TypeError: unsupported operand type(s) for @: 'numpy.ndarray' and 'Tensor'

In [30]:
TM = torch.tensor(Weak.M)
TV = torch.tensor(Weak.V)
TVM = TV@TM
TVM.dtype

torch.float64

In [22]:
examples = next(iter(trainloader))
yhat = mlp(examples[0])

soft= torch.nn.Softmax(dim=1)
p = soft(yhat)

In [29]:
p.dtype

torch.float32

In [28]:
TV.T
torch.log(TVM@p.T)

RuntimeError: expected m1 and m2 to have the same dtype, but got: double != float

In [ ]:
class FBLoss(nn.Module):
    def __init__(self, M, V):
        super(FBLoss, self).__init__()
        self.softmax = torch.nn.Softmax(dim=1)
        self.M = torch.tensor(M)
        self.V = torch.tensor(V)
        self.VM = V@M
        
    def forward(self,out,z):
        p = self.softmax(out)
        #Loss L(z,f) = z'L(f) = z'V'phi(VMf)
        L = -(self.V.T@torch.log(self.VM@p))[z]

        return L

In [ ]:
'''
import pickle
Results_mnist_ce = [overall_results,overall_models]
f = open("Experimental_results/EM.pkl","wb")
pickle.dump(Results_mnist_ce,f)
f.close()
'''

In [ ]:
import matplotlib.pyplot as plt

tr_loss = np.array(overall_results[0]['train_loss'])
tr_acc = np.array(overall_results[0]['train_acc'])
te_acc = np.array(overall_results[0]['test_acc'])
for i in range(1):
    tr_loss = np.vstack((tr_loss,overall_results[i+1]['train_loss']))
    tr_acc = np.vstack((tr_acc,overall_results[i+1]['train_acc']))
    te_acc = np.vstack((te_acc,overall_results[i+1]['test_acc']))
plt.plot(np.mean(tr_loss,0))
plt.plot(np.mean(tr_loss,0)+np.std(tr_loss,0),'g--')
plt.plot(np.mean(tr_loss,0)-np.std(tr_loss,0),'g--')

In [ ]:
plt.plot(np.mean(te_acc,0))
plt.plot(np.mean(te_acc,0)+np.std(te_acc,0),'g--')
plt.plot(np.mean(te_acc,0)-np.std(te_acc,0),'g--')

In [ ]:
plt.plot(np.mean(tr_acc,0))
plt.plot(np.mean(tr_acc,0)+np.std(tr_acc,0),'g--')
plt.plot(np.mean(tr_acc,0)-np.std(tr_acc,0),'g--')